In [ ]:
!pip install gensim   # установим библиотеку если еще не

# Task 2 - Intruder detection

## Краткое описание задачи

Будем решать задачу идентификации взломщика по его поведению в сети Интернет. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду.

В этом соревновании будем решать похожую задачу: алгоритм будет анализировать последовательность из нескольких веб-сайтов, посещенных подряд одним и тем же человеком, и определять, это нормальный юзер или взломщик.

В обучающей выборке train_sessions.csv признаки site_i – это индексы посещенных сайтов (расшифровка дана в pickle-файле со словарем site_dic.pkl).
Признаки time_j – время посещения сайтов site_j.

Целевой признак target – факт того, что сессия принадлежит юзеру (то есть что именно юзер, а не взломщик ходил по всем этим сайтам).

Задача – сделать прогнозы для сессий в тестовой выборке (test_sessions.csv), определить, принадлежат ли они реальному юзеру.

In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from gensim.models import word2vec

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
SEED = 42

In [ ]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('data/intruder_detection/train_sessions.csv')#,index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на кусочек обучающей выборки
train_df.head()

In [ ]:
train_df.target.sum()/train_df.shape[0]  # классы вообще не сбалансированы

In [ ]:
sites = ['site%s' % i for i in range(1, 11)]
#заменим nan на 0
train_df[sites] = train_df[sites].fillna(0).astype('int').astype('str')

#создадим псевдотексты необходимые для обучения word2vec
train_df['list'] = train_df['site1']

for s in sites[1:]:
    train_df['list'] = train_df['list']+","+train_df[s]

train_df['list_w'] = train_df['list'].apply(lambda x: x.split(','))

In [ ]:
# В нашем случае предложение это набор сайтов, которые посещал пользователь
# нам необязательно переводить айдишники сайтов в названия, т.к. алгоритм будем выявлять их взаимосвязь друг с другом
train_df['list_w'].iloc[1]

In [ ]:
# обучим нашу модель на всех данных 
# с размером окна=3(длина предложения 10 слов) и итоговыми векторами размерности 300, 
# параметр workers отвечает за количество ядер

data = train_df

In [ ]:
%%time
vec_size = 300
window_size = 3
n_workers = 4
model = word2vec.Word2Vec(data['list_w'], size=vec_size, window=window_size, workers=n_workers)

In [ ]:
#создадим словарь со "словами" и соответствующими им векторами
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

В данной задаче под "словом" подразумевается айдишник сайта, а под "предложением" - последовательность посещения сайтов

Т.к. сейчас мы каждому "слову" сопоставили вектор, то нужно решить что сопоставить целому "предложению" из слов.
Один из возможных вариантов это просто усреднить все слова в предложении и получить некоторый смысл всего предложения (если слова нет в тексте, то берем нулевой вектор).

In [ ]:
def compute_mean_vectors(w2v_dict, data):
        
    return np.array([
                        np.mean([w2v_dict[w] for w in sentence if w in w2v_dict] 
                        or [np.zeros(vec_size)], axis=0)
                    for sentence in data
                    
    ])


In [ ]:
data_mean = compute_mean_vectors(w2v, train_df['list_w'].values)

In [ ]:
data_mean.shape

Разобъем выборку на обучающую и валидационную

In [ ]:
X = data_mean
y = train_df.target.values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=SEED)

Применим сверху LogisticRegression.

In [ ]:
%%time
lr = LogisticRegression(C=1, random_state=SEED, n_jobs=-1)

# Обучение
lr.fit(X_train, y_train)

# прогноз для валидационной выборки
y_pred = lr.predict_proba(X_valid)[:, 1]

In [ ]:
# считаем метрики
score = roc_auc_score(y_valid, y_pred)

print('ROC-AUC:', score)

### Попробуем взвесить вектора с idf весами

Попробуем улучшить результаты.

Теперь вместо обычного среднего, чтобы учесть частоту с которой слово встречается в тексте, возьмем взвешенное среднее. В качестве весов возьмем idf меру слова.  

Idf это инверсия частоты, с которой некоторое слово встречается в других документах. Учёт idf уменьшает вес широкоупотребительных слов и увеличивает вес более уникальных слов, которые могут достаточно точно указать на то к какому классу относится текст. В нашем случае, кому принадлежит последовательность посещенных сайтов.
$$idf(w,D)=log \frac{|D|}{|{\{d \in D | w \in d\}}|}$$
где $|D|$ - общее число документов, $\{d \in D | w \in d\}$ - число документов из $D$, в которых встречается слово $w$.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [ ]:
def tfidf_weighted_vectors(w2v_dict, data):
        
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(data)
    
    max_idf = max(tfidf.idf_)
    word2weight = defaultdict(
                                lambda: max_idf,
                                [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
                    )
    
    return np.array([
                        np.mean([w2v_dict[w] * word2weight[w] for w in sentence if w in w2v_dict] 
                        or [np.zeros(vec_size)], axis=0)
                    for sentence in data
                    
    ])


In [ ]:
%%time
data_tfidf_weighted = tfidf_weighted_vectors(w2v, train_df['list_w'].values)

In [ ]:
data_tfidf_weighted.shape

Опять разобъем выборку на обучающую и валидационную

In [ ]:
X = data_tfidf_weighted
y = train_df.target.values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=SEED)

Проверим изменилось ли качество LogisticRegression.

In [ ]:
## считается долго (3 мин) - метрики чуть лучше
%%time
lr = LogisticRegression(C=1, random_state=SEED, n_jobs=-1)

# Обучение
lr.fit(X_train, y_train)

# прогноз для валидационной выборки
y_pred = lr.predict_proba(X_valid)[:, 1]

In [ ]:
# считаем метрики
score = roc_auc_score(y_valid, y_pred)

print('ROC-AUC:', score)

Видим, что качество работы улучшилось.

Значит взвешенное среднее помогает немного лучше отразить смысл всего предложения через word2vec, нежели простое усреднение

# Task3 - Предсказание популярности статьи на Хабре по ее содержанию

Испробуем мощь word2vec на статьях Хабра.

In [ ]:
import re
import pandas as pd
from gensim.models import word2vec
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
SEED = 42

Загрузим уже частично обработанную выборку

В частности, таргет представлен в прологарифмированной форме. 

**Зачем ?** 

In [ ]:
!wget https://t.bk.ru/7Eyyw/train_small.csv 
!mv train_small.csv data/habr_task

In [ ]:
%%time
data_habr = pd.read_csv('data/habr_task/train_small.csv')

In [ ]:
data_habr.shape

In [ ]:
data_habr.head()

Будем обучать модель на всем содержании статьи. Для этого совершим некоторые преобразования над текстом.

Напишем функцию, которая будет преобразовывать текстовую статью в лист из слов необходимый для обучения Word2Vec.
Функция получает строку, в которой содержится весь текстовый документ.

1) Сначала функция будет удалять все символы кроме букв верхнего и нижнего регистра;

2) Затем преобразовывает слова к нижнему регистру и возвращает список слов

In [ ]:
def review_to_wordlist(review):
    review_text = re.sub("[^а-яА-Яa-zA-Z]"," ", review)
    words = review_text.lower().split()
    return(words)

In [ ]:
# если есть nan,преобразуем их к строке
data_habr['content_clear'] = data_habr['content'].apply(str)

И наконец преобразуем контент статьи к списку слов

In [ ]:
%%time
data_habr['content_clear'] = data_habr['content_clear'].apply(review_to_wordlist)

In [ ]:
%%time
## approx 3 min
model = word2vec.Word2Vec(data_habr['content_clear'], size=300, window=5, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

Считается достаточно долго, поэтому сохраним модель и будем подгружать ее впоследствии

In [ ]:
%%time
model.save("data/habr_task/word2vec.model")

подгрузим модель

In [ ]:
!wget https://t.bk.ru/8B6cR/word2vec.model.trainables.syn1neg.npy
!mv word2vec.model.trainables.syn1neg.npy data/habr_task/

!wget https://t.bk.ru/UDb5P/word2vec.model.wv.vectors.npy
!mv word2vec.model.wv.vectors.npy data/habr_task/

In [ ]:
%%time
model = word2vec.Word2Vec.load("data/habr_task/word2vec.model")
w2v = dict(zip(model.wv.index2word, model.wv.vectors))

## Теперь можно посмотреть, чему выучилась модель

In [ ]:
model.wv.most_similar(positive=['привет'])

In [ ]:
model.wv.most_similar(positive=['python'])

In [ ]:
model.wv.most_similar(positive=['machine', 'learning'])

In [ ]:
model.wv.most_similar(positive=['iphone'])

Ну что-то адекватное

## Давайте используем полученные вектора для предсказания рейтинга статьи

In [ ]:
vec_size = 300

In [ ]:
def compute_mean_vectors(w2v_dict, data):
        
    return np.array([
                        np.mean([w2v_dict[w] for w in sentence if w in w2v_dict] 
                        or [np.zeros(vec_size)], axis=0)
                    for sentence in data
                    
    ])


In [ ]:
%%time
## используем простое усреднение
data_mean = compute_mean_vectors(w2v, data_habr['content_clear'])

In [ ]:
X = data_mean
y = data_habr['favs_lognorm']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=SEED)

In [ ]:
%%time
ridge = Ridge(alpha=1, random_state=SEED)

# Обучение
ridge.fit(X_train, y_train)

# прогноз для валидационной выборки
y_pred = ridge.predict(X_valid)
print('MSE на валидации', mean_squared_error(y_valid, y_pred))

y_pred_train = ridge.predict(X_train)
print('MSE на трейне', mean_squared_error(y_train, y_pred_train))
